In [1]:
import pickle

from datasets import ClassLabel, Dataset, concatenate_datasets, load_dataset

from divide_and_conquer_sentiment.aggregation import MLP, MLPAggregator
from divide_and_conquer_sentiment.dataloaders import load_kaggle_dataset
from divide_and_conquer_sentiment.subprediction import ABSASubpredictor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TRAIN_BATCH_SIZE = 256

# Read & prepare dataset


In [4]:
twitter_airlines_dataset = load_kaggle_dataset(
    "crowdflower/twitter-airline-sentiment",
    {"text": "text", "airline_sentiment": "label"},
    val_test_perc=(0.1, 0.2),
    seed=42,
)

Dataset URL: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment


/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/kaggle/api_client.py:165: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  response_data.getheaders())


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/14640 [00:00<?, ? examples/s]

Map:   0%|          | 0/14640 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/14640 [00:00<?, ? examples/s]

In [5]:
amazon_headphones_dataset = load_kaggle_dataset(
    "mdwaquarazam/headphone-dataset-review-analysis",
    {"COMMENTS": "text", "RATINGS": "label"},
    val_test_perc=(0.1, 0.2),
    seed=42,
)

Dataset URL: https://www.kaggle.com/datasets/mdwaquarazam/headphone-dataset-review-analysis


/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/kaggle/api_client.py:165: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  response_data.getheaders())


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1604 [00:00<?, ? examples/s]

Map:   0%|          | 0/1604 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1604 [00:00<?, ? examples/s]

In [6]:
SST_LABEL_MAP = {0: 0, 1: 0, 2: 1, 3: 2, 4: 2}


def map_sst_label(x):
    x["label"] = SST_LABEL_MAP[x["label"]]
    return x


sst_dataset = (
    load_dataset("SetFit/sst5")
    .remove_columns(["label_text"])
    .map(map_sst_label)
    .cast_column("label", ClassLabel(names=["negative", "neutral", "positive"]))
)

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
train_dataset = concatenate_datasets(
    [twitter_airlines_dataset["train"], amazon_headphones_dataset["train"], sst_dataset["train"]]
)
val_dataset = concatenate_datasets(
    [twitter_airlines_dataset["val"], amazon_headphones_dataset["val"], sst_dataset["validation"]]
)
test_dataset = concatenate_datasets(
    [twitter_airlines_dataset["test"], amazon_headphones_dataset["test"], sst_dataset["test"]]
)

# Train model

In [8]:
!spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 30.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [49]:
subpredictor = ABSASubpredictor.from_pretrained(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
    spacy_model="en_core_web_lg",
)
mlp = MLP(input_size=4, output_size=3, hidden_layer_sizes=(128, 64), lr=0.001)
aggregator = MLPAggregator(mlp)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity
INFO: GPU available: True (mps), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (mps), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
train_subpreds = subpredictor.predict(train_dataset["text"])
val_subpreds = subpredictor.predict(val_dataset["text"])

In [70]:
train_dataset = concatenate_datasets([train_dataset, Dataset.from_dict({"subpreds": train_subpreds})], axis=1)
val_dataset = concatenate_datasets([val_dataset, Dataset.from_dict({"subpreds": val_subpreds})], axis=1)

Flattening the indices:   0%|          | 0/19914 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/19914 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2726 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2726 [00:00<?, ? examples/s]

In [71]:
with open("train_dataset_subpreds.pkl", "wb") as handle:
    pickle.dump(train_dataset, handle)

with open("val_dataset_subpreds.pkl", "wb") as handle:
    pickle.dump(val_dataset, handle)

In [9]:
with open("train_dataset_subpreds.pkl", "rb") as handle:
    train_dataset = pickle.load(handle).with_format("torch")

with open("val_dataset_subpreds.pkl", "rb") as handle:
    val_dataset = pickle.load(handle).with_format("torch")

In [50]:
aggregator.train(train_dataset, val_dataset)

/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/lightning/pytorch/loops/utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
INFO: 
  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | layers | ModuleList | 11.3 K | train
----------------------------------------------
11.3 K    Trainable params
0         Non-trainable params
11.3 K    Total params
0.045     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type       | Params | Mode 
----------------------------------------------
0 | layers | ModuleList | 11.3 K | train
----------------------------------------------
11.3 K    Trainable params
0         Non-trainable params
11.3 K    Total params
0.045     Total estimated model params size (MB)
4         Modules in 

Training: |          | 0/? [00:00<?, ?it/s]

Process Process-auto_conversion:
Process Process-auto_conversion:
INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...
INFO:lightning.pytorch.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/transformers/safetensors_conversion.py", line 84, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
  File "/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/

NameError: name 'exit' is not defined